# **RAVIR**         

Axel Romero       
*Deep Learning for Medical Image Analysis*

In [1]:
from source.data import RAVIRDataset
from source.segravir import SegRAVIRModel

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchview import draw_graph
from torchsummary import summary

In [2]:
model = SegRAVIRModel()

summary(model, (3, 256, 256))

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 16, 256, 256]        448
|    └─ResidualBlock: 2-2                [-1, 16, 256, 256]        --
|    |    └─Conv2d: 3-1                  [-1, 16, 256, 256]        2,320
|    |    └─BatchNorm2d: 3-2             [-1, 16, 256, 256]        32
|    |    └─Dropout: 3-3                 [-1, 16, 256, 256]        --
|    |    └─Conv2d: 3-4                  [-1, 16, 256, 256]        2,320
|    |    └─BatchNorm2d: 3-5             [-1, 16, 256, 256]        32
|    |    └─Dropout: 3-6                 [-1, 16, 256, 256]        --
|    |    └─Identity: 3-7                [-1, 16, 256, 256]        --
|    └─Conv2d: 2-3                       [-1, 32, 128, 128]        4,640
|    └─ResidualBlock: 2-4                [-1, 32, 128, 128]        --
|    |    └─Conv2d: 3-8                  [-1, 32, 128, 128]        9,248
| 

Layer (type:depth-idx)                   Output Shape              Param #
├─ModuleList: 1                          []                        --
|    └─Conv2d: 2-1                       [-1, 16, 256, 256]        448
|    └─ResidualBlock: 2-2                [-1, 16, 256, 256]        --
|    |    └─Conv2d: 3-1                  [-1, 16, 256, 256]        2,320
|    |    └─BatchNorm2d: 3-2             [-1, 16, 256, 256]        32
|    |    └─Dropout: 3-3                 [-1, 16, 256, 256]        --
|    |    └─Conv2d: 3-4                  [-1, 16, 256, 256]        2,320
|    |    └─BatchNorm2d: 3-5             [-1, 16, 256, 256]        32
|    |    └─Dropout: 3-6                 [-1, 16, 256, 256]        --
|    |    └─Identity: 3-7                [-1, 16, 256, 256]        --
|    └─Conv2d: 2-3                       [-1, 32, 128, 128]        4,640
|    └─ResidualBlock: 2-4                [-1, 32, 128, 128]        --
|    |    └─Conv2d: 3-8                  [-1, 32, 128, 128]        9,248
| 